# 数据处理 Dataset

TensorFlow 提供了一种内置的 API——`Dataset`，使得我们可以很容易地就利用输入管道的方式输入数据。在[这篇教程](https://blog.csdn.net/dQCFKyQDXYm3F8rB0/article/details/79342369)中，介绍如何创建和使用输入管道以及如何高效地向模型输入数据。

文章中的代码：https://github.com/FrancescoSaverioZuppichini/Tensorflow-Dataset-Tutorial/blob/master/dataset_tutorial.ipynb

In [1]:
import tensorflow as tf
import numpy as np

C:\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
x = np.random.sample((100,2))
# make a dataset from a numpy array
dataset = tf.data.Dataset.from_tensor_slices(x)

data_iter = dataset.make_one_shot_iterator()
el = data_iter.get_next()

with tf.Session() as sess:
    print(sess.run(el))

[0.11586518 0.89992529]


## 我们也可以传递多个 numpy 数组，最典型的例子是当数据被划分为特征和标签的时候：

In [3]:
# using two numpy arrays
features, labels = (np.random.sample((100,2)), np.random.sample((100,1)))
dataset = tf.data.Dataset.from_tensor_slices((features,labels))

data_iter = dataset.make_one_shot_iterator()
el = data_iter.get_next()

with tf.Session() as sess:
    print(sess.run(el))

(array([0.77302352, 0.2778674 ]), array([0.73686871]))


## 我们也可以用一些张量初始化数据集

In [4]:
# using a tensor
dataset = tf.data.Dataset.from_tensor_slices(tf.random_uniform([100, 2]))

data_iter = dataset.make_initializable_iterator()
el = data_iter.get_next()

with tf.Session() as sess:
    sess.run(data_iter.initializer)
    print(sess.run(el))

[0.44105017 0.95432734]


## 从placeholder中载入

如果我们想动态地改变Dataset中的数据，使用这种方式是很有用的。

In [5]:
# using a placeholder
x = tf.placeholder(tf.float32, shape=[None,2])
dataset = tf.data.Dataset.from_tensor_slices(x)

data = np.random.sample((100,2))

data_iter = dataset.make_initializable_iterator()
el = data_iter.get_next()

with tf.Session() as sess:
    sess.run(data_iter.initializer, feed_dict={ x: data })
    print(sess.run(el))

[0.38787663 0.8495356 ]


## 从 generator 载入

我们也可以从 generator 中初始化一个 Dataset。当一个数组中元素长度不相同时，使用这种方式处理是很有效的。（例如一个序列）

In [6]:
# from generator
sequence = np.array([[1],[2,3],[3,4]])

def generator():
    for el in sequence:
        yield el

dataset = tf.data.Dataset().from_generator(generator,
                                           output_types=tf.float32, 
                                           output_shapes=[tf.float32])
data_iter = dataset.make_initializable_iterator()
el = data_iter.get_next()

with tf.Session() as sess:
    sess.run(data_iter.initializer)
    print(sess.run(el))

[1.]


在这种情况下，你还需要指定数据的类型和大小以创建正确的tensor

# 创建一个迭代器

我们已经知道了如何创建数据集，但是如何从中获取数据呢？我们需要使用一个 Iterator 遍历数据集并重新得到数据真实值。有四种形式的迭代器。

## One shot Iterator

这是最简单的迭代器，下面给出第一个例子：

In [7]:
# initializable iterator to switch between data
EPOCHS = 10

x, y = tf.placeholder(tf.float32, shape=[None,2]), tf.placeholder(tf.float32, shape=[None,1])
dataset = tf.data.Dataset.from_tensor_slices((x, y))

train_data = (np.random.sample((100,2)), np.random.sample((100,1)))
test_data = (np.array([[1,2]]), np.array([[0]]))

data_iter = dataset.make_initializable_iterator()
features, labels = data_iter.get_next()

with tf.Session() as sess:
#     initialise iterator with train data
    sess.run(data_iter.initializer, feed_dict={ x: train_data[0], y: train_data[1]})
    for _ in range(EPOCHS):
        sess.run([features, labels])
#     switch to test data
    sess.run(data_iter.initializer, feed_dict={ x: test_data[0], y: test_data[1]})
    print(sess.run([features, labels]))

[array([1., 2.], dtype=float32), array([0.], dtype=float32)]


In [18]:
# Reinitializable iterator to switch between Datasets
EPOCHS = 10
# making fake data using numpy
train_data = (np.random.sample((100,2)), np.random.sample((100,1)))
test_data = (np.random.sample((10,2)), np.random.sample((10,1)))
# create two datasets, one for training and one for test
train_dataset = tf.data.Dataset.from_tensor_slices(train_data)
test_dataset = tf.data.Dataset.from_tensor_slices(test_data)
# create a iterator of the correct shape and type
data_iter = tf.data.Iterator.from_structure(train_dataset.output_types,
                                           train_dataset.output_shapes)
features, labels = data_iter.get_next()
# create the initialisation operations
train_init_op = data_iter.make_initializer(train_dataset)
test_init_op = data_iter.make_initializer(test_dataset)

with tf.Session() as sess:
    sess.run(train_init_op) # switch to train dataset
    for _ in range(EPOCHS):
        sess.run([features, labels])
    sess.run(test_init_op) # switch to val dataset
    print(sess.run([features, labels]))

[array([0.22875896, 0.10518428]), array([0.19003748])]


In [19]:
# BATCHING
BATCH_SIZE = 4
x = np.random.sample((100,2))
# make a dataset from a numpy array
dataset = tf.data.Dataset.from_tensor_slices(x).batch(BATCH_SIZE)

data_iter = dataset.make_one_shot_iterator()
el = data_iter.get_next()

with tf.Session() as sess:
    print(sess.run(el))

[[0.3916214  0.64185074]
 [0.61914613 0.21525406]
 [0.17707292 0.94198672]
 [0.72675197 0.09594369]]


In [3]:
# REPEAT
BATCH_SIZE = 4
x = np.array([[1],[2],[3],[4]])
# make a dataset from a numpy array
dataset = tf.data.Dataset.from_tensor_slices(x)
dataset = dataset.repeat()

data_iter = dataset.make_one_shot_iterator()
el = data_iter.get_next()

with tf.Session() as sess:
#     this will run forever
    while True:
        print(sess.run(el))

[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]


[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]


[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]


[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]
[2]
[3]
[4]
[1]


KeyboardInterrupt: 

In [4]:
# MAP
x = np.array([[1],[2],[3],[4]])
# make a dataset from a numpy array
dataset = tf.data.Dataset.from_tensor_slices(x)
dataset = dataset.map(lambda x: x*2)

data_iter = dataset.make_one_shot_iterator()
el = data_iter.get_next()

with tf.Session() as sess:
#     this will run forever
        for _ in range(len(x)):
            print(sess.run(el))

[2]
[4]
[6]
[8]


In [22]:
# SHUFFLE
BATCH_SIZE = 4
x = np.array([[1],[2],[3],[4]])
# make a dataset from a numpy array
dataset = tf.data.Dataset.from_tensor_slices(x)
dataset = dataset.shuffle(buffer_size=100)
dataset = dataset.batch(BATCH_SIZE)

data_iter = dataset.make_one_shot_iterator()
el = data_iter.get_next()

with tf.Session() as sess:
    print(sess.run(el))

[[2]
 [4]
 [3]
 [1]]


In [5]:
# how to pass the value to a model
EPOCHS = 10
BATCH_SIZE = 16
# using two numpy arrays
features, labels = (np.array([np.random.sample((100,2))]), 
                    np.array([np.random.sample((100,1))]))

dataset = tf.data.Dataset.from_tensor_slices((features,labels)).repeat().batch(BATCH_SIZE)

data_iter = dataset.make_one_shot_iterator()
x, y = data_iter.get_next()

# make a simple model
net = tf.layers.dense(x, 8, activation=tf.tanh) # pass the first value from iter.get_next() as input
net = tf.layers.dense(net, 8, activation=tf.tanh)
prediction = tf.layers.dense(net, 1, activation=tf.tanh)

loss = tf.losses.mean_squared_error(prediction, y) # pass the second value from iter.get_net() as label
train_op = tf.train.AdamOptimizer().minimize(loss)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(EPOCHS):
        _, loss_value = sess.run([train_op, loss])
        print("Iter: {}, Loss: {:.4f}".format(i, loss_value))

Iter: 0, Loss: 0.1377
Iter: 1, Loss: 0.1322
Iter: 2, Loss: 0.1272
Iter: 3, Loss: 0.1228
Iter: 4, Loss: 0.1190
Iter: 5, Loss: 0.1156
Iter: 6, Loss: 0.1127
Iter: 7, Loss: 0.1102
Iter: 8, Loss: 0.1081
Iter: 9, Loss: 0.1064


In [2]:
# Wrapping all together -> Switch between train and test set
EPOCHS = 10
BATCH_SIZE = 16

x, y = tf.placeholder(tf.float32, shape=[None,2]), tf.placeholder(tf.float32, shape=[None,1])
dataset = tf.data.Dataset.from_tensor_slices((x, y)).batch(BATCH_SIZE).repeat()

# using two numpy arrays
train_data = (np.random.sample((100,2)), np.random.sample((100,1)))
test_data = (np.random.sample((20,2)), np.random.sample((20,1)))

n_batches = len(train_data[0]) // BATCH_SIZE

iter = dataset.make_initializable_iterator()
features, labels = iter.get_next()
# make a simple model
net = tf.layers.dense(features, 8, activation=tf.tanh) # pass the first value from iter.get_next() as input
net = tf.layers.dense(net, 8, activation=tf.tanh)
prediction = tf.layers.dense(net, 1, activation=tf.tanh)

loss = tf.losses.mean_squared_error(prediction, labels) # pass the second value from iter.get_net() as label
train_op = tf.train.AdamOptimizer().minimize(loss)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    # initialise iterator with train data
    sess.run(iter.initializer, feed_dict={ x: train_data[0], y: train_data[1]})
    print('Training...')
    for i in range(EPOCHS):
        tot_loss = 0
        for _ in range(n_batches):
            _, loss_value = sess.run([train_op, loss])
            tot_loss += loss_value
        print("Iter: {}, Loss: {:.4f}".format(i, tot_loss / n_batches))
    # initialise iterator with test data
    sess.run(iter.initializer, feed_dict={ x: test_data[0], y: test_data[1]})
    print('Test Loss: {:4f}'.format(sess.run(loss)))


Training...
Iter: 0, Loss: 0.4637
Iter: 1, Loss: 0.3414
Iter: 2, Loss: 0.2971
Iter: 3, Loss: 0.2280
Iter: 4, Loss: 0.2002
Iter: 5, Loss: 0.1622
Iter: 6, Loss: 0.1336
Iter: 7, Loss: 0.1253
Iter: 8, Loss: 0.1030
Iter: 9, Loss: 0.0996
Test Loss: 0.089990


In [ ]:
|